## Mathematik für Biologiestudierende II

Sommersemester 2024

18.06.2024

&copy; 2024 Prof. Dr. Rüdiger W. Braun 

# Evaluation

Vorrangige Themen waren

* Vorlesungsaufzeichnungen
* Permanenz der Online-Übungen
* Klausurvorbereitung

In [ ]:
import numpy as np
import pandas as pd
from scipy import stats
import statsmodels.formula.api as smf
import seaborn as sns
sns.set_theme()

# Lineare Modelle

#### Themen heute

* Vorhersagen
* Konfidenzintervalle für den Mittelwert und für die Einzelbeobachtung
* mehr als eine erklärende Variable
* irrelevante erklärende Variable erkennen

# Vorhersagen (prediction)

In [ ]:
df = pd.read_csv('galton.csv')
df.head()

Aufbereitung eines Datensatzes von Galton.  Die Aufbereitung stammt aus den Begleitdaten zum Buch "Linear Models with Python" von Faraway

In [ ]:
formel = 'childHeight ~ father'

* In dieser Formel ist `father` die *erklärende* und `Childheight` die *erkärte* Variable
* Die erklärende Variable heißt auch *exogen*, die erklärte *endogen* 

In [ ]:
modell = smf.ols(formel, df)
res = modell.fit()
res.summary()

In [ ]:
sns.regplot(df, x='father', y='childHeight');

* Aufgabe:  Wie groß ist im Mittel der Sohn, wenn der Vater 69.8 Zoll groß ist?

Erste Lösung:

* Wir lesen ab

In [ ]:
m = 0.4465
b = 38.36

In [ ]:
x = 69.8
y = m*x + b
y

Zweite Lösung:

* Wir verwenden die Methode `get_prediction`
* Dazu müssen die Daten der erklärenden Variablen in einen DataFrame geschrieben werden

In [ ]:
anfrage = pd.DataFrame()
anfrage['father'] = [0, 69.8]   
#  rechte Seite ist auch dann ein array, wenn nur ein Wert berechnet werden soll
anfrage

In [ ]:
res.get_prediction(anfrage).summary_frame()

* `mean`:  Wert, der für den Sohn im Mittel zu erwarten ist
* `mean` bei 0 ist das Intercept
* `mean_se`:  Standardabweichung für `mean`

* die vier anderen Werte sind untere bzw. obere Grenzen für Konfidenzintervalle
* `mean_ci` ist das Konfidenzintervall für den mittleren zu erwartenden Wert
* `obs_ci` ist das Konfidentintervall für den individuelle beobachteten Wert (engl: "observed")

* `mean_ci_lower` und `mean_ci_upper` begrenzen die blaue Kurve in dem `regplot`
* `obs_ci_lower` und `obs_ci_upper` begrenzen einen Bereich, der 95% der Beobachtungen enthält

* wir malen den mal hin

In [ ]:
anfrage = pd.DataFrame()
anfrage['father'] = np.arange(625, 775) / 10
vorhersage = res.get_prediction(anfrage).summary_frame()
vorhersage

In [ ]:
ax = sns.scatterplot(df, x='father', y='childHeight')
sns.lineplot(x=anfrage.father, y=vorhersage['mean'], ax=ax)
sns.lineplot(x=anfrage.father, y=vorhersage.obs_ci_lower, ax = ax, color='orange')
sns.lineplot(x=anfrage.father, y=vorhersage.obs_ci_upper, ax=ax, color='orange');

Die orangen Linien sind die untere bzw. obere Vertrauensgrenze für die Einzelbeobachtungen

Anderes Konfidenzniveau $1-\alpha$

In [ ]:
res.get_prediction(anfrage).summary_frame(alpha=0.02)

## Beispiel:  Fische

* Fische werden gezüchtet.  In den ersten 24 Monaten wurden die folgenden Daten erhoben
* Diesen Daten werden benutzt, um das Wachstum der nächsten Generation zu prognostizieren

In [ ]:
rng = np.random.default_rng(123)
N = 70

In [ ]:
df = pd.DataFrame()
df['Monat'] = rng.choice(np.arange(4, 25), size=N)
df['Höhe'] = 4.5*df.Monat + stats.norm(0, 2.2).rvs(size=N, random_state=rng)
df['Gewicht'] = 65*df.Monat + 4*df.Höhe + stats.norm(0, 50).rvs(size=N, random_state=rng)
#df.to_csv('fische.csv', index=False)

In [ ]:
df = pd.read_csv('fische.csv')
df.describe()

* Gewicht in g
* Höhe in mm

* Ein Züchter hat 1200 Fische in seinen Teichen, die alle gleichzeitig geschlüpft sind
* Frage:  Konfidenzintervall für das Gesamtgewicht dieser Fische nach 18 Monaten zum Konfidenzniveau 95%?
* Frage:  Wie muss das Netz gewählt werden, um nach 18 Monaten 97.5% der Fische zu fangen?

In [ ]:
formel1 = 'Gewicht ~ Monat'
modell1 = smf.ols(formel1, df)

In [ ]:
res = modell1.fit()
res.summary()

In [ ]:
anfrage = pd.DataFrame()
anfrage['Monat'] = [18]
res.get_prediction(anfrage).summary_frame()

* untere Vertrauensgrenze für das Gesamtgewicht von 1200 Fischen in kg:

In [ ]:
1200 * 1493 / 1000

* obere Vertrauensgrenze für das Gesamtgewicht von 1200 Fischen in kg:

In [ ]:
1200 *  1523 / 1000

Mit 97.5% Sicherheit werden mindestens 1791 kg Fisch geerntet

In [ ]:
formel2 = 'Höhe ~ Monat'
modell2 = smf.ols(formel2, df)
res = modell2.fit()
res.summary()

In [ ]:
res.get_prediction(anfrage).summary_frame()

Um 97.5% der Fische zu fangen, muss das Netz so beschaffen sein, dass ein Fisch der Höhe 76.9mm nicht hindurch schlüpft

# Mehrere erklärende Variablen

### Lineares Modell mit einer erklärten und mehreren erklärenden Variablen

$$
   y = m_1 \cdot x_1 + m_2 \cdot x_2 + \dots + m_n \cdot x_n + b
$$

$y$ ist die erklärte und die $x_i$ sind die erklärenden Variablen

Beispiel:  Körpergröße der Söhne hängt von der Körpergröße von Vater und Mutter ab

In [ ]:
df = pd.read_csv('galton.csv')

In [ ]:
formel = 'childHeight ~ father + mother'

Diese Formel hat 3 Unbekannte:

* den Koeffizienten von `father`
* den Koeffizienten von `mother`
* den Ordinatenabschnitt

In [ ]:
modell = smf.ols(formel, df)

In [ ]:
res = modell.fit()

In [ ]:
res.summary()

* Regressiongleichung:  childHeight = 0.4176 * father + 0.3288 * mother + 19.3128
* alle drei Koeffizienten haben statistisch signifikanten Einfluss

Zum Vergleich

In [ ]:
formel2 = 'childHeight ~ father'
modell2 = smf.ols(formel2, df)
res = modell2.fit()

In [ ]:
res.summary()

* Regressionsgleichung  childHeight = 0.4465 * father + 38.3626
* beide Koeffizienten haben statistisch signifikanten Einfluss

* Das erste Modell ist genauer, denn dort ist der Wert von `R-squared` höher

Füge einen irrelevanten Term in den Datensatz ein

In [ ]:
rng = np.random.default_rng()
df['Kontonummer'] = rng.integers(1000, 99999, size=481)
df.head()

In [ ]:
formel3 = 'childHeight ~ father + mother + Kontonummer'
modell3 = smf.ols(formel3, df)
res = modell3.fit()

In [ ]:
res.summary()

* Die Koeffizienten sind etwas verändert gegenüber dem ersten Modell 
* Der Unterschied ist nicht signifikant
* Wert von `R-squared` unverändert gegenüber dem ersten Modell

* Von den hier vorgestellten Modellen ist das erste das beste